In [1]:
from transformers import AutoTokenizer

In [46]:
# model_name = 'meta-llama/Llama-2-7b'
model_name = 'meta-llama/Llama-3.1-8B'
# model_name = 'meta-llama/Meta-Llama-3-8B'
# model_name = 'gpt2'

In [47]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [48]:
tokenizer.special_tokens_map

{'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>'}

In [49]:
tokenizer.padding_side

'right'

### `tokenizer.pad_token = tokenizer.eos_token`

- training
    - sent1: "Hello, how are you?"
        - `["Hello,", " how", " are", " you?", <eos>]`
    - sent2: "I am fine."
        - `["I", " am", "fine.", <eos>]`
- padding for batch processing
    - sent1: `["Hello,", " how", " are", " you?", <eos>]`
    - sent2: `["I", " am", "fine.", <eos>, <pad>]`
- pad_token = eos_token
    - sent1: `["Hello,", " how", " are", " you?", <eos>]`
    - sent2: `["I", " am", "fine.", <eos>, <eos>]`

In [50]:
sent1 = "Hello, how are you?"
sent2 = "I am fine."

In [51]:
tokenizer.encode(sent1)

[128000, 9906, 11, 1268, 527, 499, 30]

In [53]:
tokenizer.tokenize(sent1)

['Hello', ',', 'Ġhow', 'Ġare', 'Ġyou', '?']

In [52]:
tokenizer.decode(128000), tokenizer.decode(30)

('<|begin_of_text|>', '?')